# 05장. 데이터 저장

3가지 데이터 관리 방법

1. 데이터 베이스 : 스크랩한 데이터를 웹사이트에 사용하거나 직접 API를 만드는 경우
2. 파일 스트림 : 하드디스크에 저장할 쉽고 빠른 방법을 찾는 경우
3. 이메일 전송 : 주기적 알림을 받거나 하루 한 번 데이터를 집계

### 미디어 파일

1. 참조를 저장
2. 파일 자체를 내려받기


In [4]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

In [2]:
html = urlopen("http://www.pythonscraping.com")
bs0bj = BeautifulSoup(html, "html.parser")
imageLocation = bs0bj.find("a", {"id": "logo"}).find("img")["src"]
urlretrieve(imageLocation, "logo.jpg")

('logo.jpg', <http.client.HTTPMessage at 0x23408062dd8>)

In [3]:
# 하나의 파일만 내려받았음

In [5]:
downloadDirectory = "downloaded"
baseUrl = "http://pythonscraping.com"

In [6]:
def getAbsoluteURL(baseUrl, source):
    if source.startswith("http://www."):
        url = "http://" + source[11:]
    elif source.startswith("http://"):
        url = source
    elif source.startswith("www."):
        url = source[4:]
        url = "http://" + source
    else:
        url = baseUrl + "/" + source
    if baseUrl not in url:
        return None
    return url

In [8]:
def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace("www.", "")
    path = path.replace(baseUrl, "")
    path = downloadDirectory + path
    directory = os.path.dirname(path)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    return path

In [9]:
# html = urlopen("http://www.pythonscraping.com")
# bsObj = BeautifulSoup(html, "html.parser")
# downloadList = bsObj.findAll(src=True)

# for download in downloadList:
#     fileUrl = getAbsoluteURL(baseUrl, download["src"])
#     if fileUrl is not None:
#         print(fileUrl)
#         urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

In [10]:
# 실행하면 만나는 모든 파일을 내려받음..! 멀웨어 조심

# csv 저장

In [11]:
import csv

In [15]:
csvFile = open("test.csv", "w+")
try:
    writer = csv.writer(csvFile)
    writer.writerow(("number", "number plus 2", "number times 2"))
    for i in range(10):
        writer.writerow( (i, i+2, i*2 ))
finally:
    csvFile.close()

### HTML테이블 => CSV

In [16]:
html = urlopen("http://en.wikipedia.org/wiki/Comparison_of_text_editors")
bsObj = BeautifulSoup(html, "html.parser")
#The main comparison table is currently the first table on the page
table = bsObj.findAll("table",{"class":"wikitable"})[0]
rows = table.findAll("tr")

csvFile = open("editors.csv", 'wt', newline='', encoding='utf-8')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
        writer.writerow(csvRow)
finally:
    csvFile.close()

## MySQL

- 관계형 데이터베이스
- http://dev.mysql.com/downloads/windows/installer/  설치 (setup type에서 Server Only 선택)


#### 기본 명령어
- CREATE DATABASE scraping
- USE scraping
- CREATE TABLE pages(id BIGINT(7) NOT NULL AUTO_INCREMENT, title VARCHAR(200), content VARCHAR(10000), created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, PRIMARY KEY(id));

- 각 열의 정의는 이름 / 변수 타입 / 추가 속성 그리고 마지막에 키를 정의해야 함

In [19]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='root', passwd='900402', db='mysql', charset='utf8')

cur = conn.cursor()
cur.execute("USE scraping")
cur.execute("SELECT * FROM pages WHERE id=1")
print(cur.fetchone())
cur.close()
conn.close()

(1, 'test page title', 'this is some test page content. It can be up to 10,000 characters long.', datetime.datetime(2017, 1, 22, 16, 15, 52))


연결 객체 ( conn )
- 데이터 베이스 연결에 관여 / 정보를 보내고 롤백을 처리하고 새 커서 객체를 만드는 역할


커서 ( cur )
- 상태 정보를 추적 / 마지막에 실행항 쿼리 결과
- cur.fetchone() 으로 함수를 호출해 정보 접근 가능
- 커서와 연결 사용을 마치면 이들을 닫아야 함..! => 연결 누수 방지